# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.metrics import mean_absolute_percentage_error, MAPE
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import make_scorer

2021-11-25 17:41:19.226329: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-25 17:41:19.226383: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# data prep

In [3]:
orig_df = pd.read_csv(
    '/home/mrstoian/code/mrstoian/btc_sent_fc/raw_data/X_y.csv', index_col=0, parse_dates=True)

In [4]:
orig_df

,n-transactions-per-block,difficulty,utxo-count,mvrv,nvt,avg-block-size,n-transactions-excluding-popular,n-unique-addresses,median-confirmation-time,miners-revenue,...,S&P U.S. TIPS 10+ Year Index (USD),S&P U.S. Treasury Bill 3-6 Month Index,S&P U.S. Treasury Bond 7-10 Year Index (TTM JPY),S&P U.S. TIPS 7-10 Year Index (USD),S&P U.S. Treasury Bond 3-5 Year Index,S&P U.S. TIPS 3-5 Year Index (USD),S&P U.S. TIPS 1-3 Year Index (USD),S&P U.S. TIPS 5-7 Year Index (USD),S&P U.S. Treasury Bill 6-9 Month Index,volume_gross
2016-06-30,1546.400000,2.094532e+11,3.973747e+07,1.835217,7.901799,0.797510,205005.000000,363625.000000,8.344444,2.633621e+06,...,283.800000,237.02,164.840000,230.2300,447.490000,183.100000,136.770,210.160000,230.05,1.015565e+07
2016-07-01,1414.325108,2.094532e+11,3.976440e+07,1.896975,7.678641,0.736557,202414.000000,356294.000000,8.166667,2.669238e+06,...,287.350000,237.02,165.270000,231.3200,447.520000,183.450000,136.870,210.810000,230.06,9.049903e+06
2016-07-02,1282.250216,2.094880e+11,3.979276e+07,1.890508,7.786408,0.675603,205482.000000,348963.000000,7.988889,2.704855e+06,...,288.245000,237.02,165.102500,231.6225,447.817500,183.500000,136.885,210.940000,230.06,6.774267e+06
2016-07-03,1150.175325,2.095228e+11,3.982112e+07,1.884042,7.894175,0.614649,208550.000000,360712.666667,7.811111,2.740471e+06,...,289.140000,237.02,164.935000,231.9250,448.115000,183.550000,136.900,211.070000,230.06,8.413963e+06
2016-07-04,1277.952597,2.095577e+11,3.984947e+07,1.877575,8.001942,0.675615,211618.000000,372462.333333,7.633333,2.625866e+06,...,290.035000,237.02,164.767500,232.2275,448.412500,183.600000,136.915,211.200000,230.06,5.634703e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-18,2054.611695,2.267415e+13,7.682603e+07,2.437422,2.814664,1.270386,275592.333333,725020.000000,7.297222,5.265043e+07,...,436.770000,252.34,204.430000,299.4800,491.070000,222.170000,159.890,266.180000,245.77,3.137042e+09
2021-11-19,2078.240876,2.267415e+13,7.692242e+07,2.423386,3.531133,1.275837,262180.666667,678741.666667,7.027778,4.966713e+07,...,439.660000,252.34,204.830000,298.6000,490.890000,221.120000,159.270,264.990000,245.77,2.119749e+09
2021-11-20,2078.240876,2.267415e+13,7.701882e+07,2.409351,4.247603,1.275837,248769.000000,632463.333333,6.758333,4.966713e+07,...,436.973333,252.34,204.843333,297.7300,490.333333,220.793333,159.150,264.383333,245.76,8.896024e+08
2021-11-21,2078.240876,2.267415e+13,7.711521e+07,2.409351,4.247603,1.275837,248769.000000,586185.000000,6.758333,4.966713e+07,...,434.286667,252.34,204.856667,296.8600,489.776667,220.466667,159.030,263.776667,245.75,7.894335e+08


# Sequencing

In [5]:
def subsample_sequence(df, length):
    """
    Given the initial dataframe `df`, return a shorter dataframe sequence of length `length`.
    This shorter sequence should be selected at random
    """
    # $CHALLENGIFY_BEGIN
    last_possible = df.shape[0] - length
    
    random_start = np.random.randint(0, last_possible)
    df_sample = df[random_start: random_start+length]
    # $CHALLENGIFY_END
    return df_sample

In [6]:
def split_subsample_sequence(df, length):
    '''Create one single random (X,y) pair'''
    # $CHALLENGIFY_BEGIN
    df_subsample = subsample_sequence(df, length)
    y_sample = df_subsample.iloc[length -1]['pm2.5']
    
    X_sample = df_subsample[0:length -1]
    X_sample = X_sample.values
    # $CHALLENGIFY_END
    return np.array(X_sample), np.array(y_sample)

In [7]:
orig_df.columns

Index(['n-transactions-per-block', 'difficulty', 'utxo-count', 'mvrv', 'nvt',
       'avg-block-size', 'n-transactions-excluding-popular',
       'n-unique-addresses', 'median-confirmation-time', 'miners-revenue',
       'mempool-growth', 'mempool-size', 'blocks-size', 'hash-rate',
       'n-transactions-total', 'avg-confirmation-time', 'nvts',
       'transaction-fees-usd', 'active_account',
       'Russell 2000 Index (RUT) - Index Value',
       'CBOE Volatility S&P 500 Index (^VIX) - Index Value',
       'S&P 500 (^SPX) - Index Value',
       'NASDAQ Composite Index (^COMP) - Index Value',
       'Dow Jones Industrial Average (^DJI) - Index Value', 'News Sentiment',
       'S&P U.S. Treasury Bill 0-3 Month Index',
       'S&P U.S. Treasury Bond 10+ Year Index',
       'S&P U.S. TIPS 5+ Year Index (USD)',
       'S&P U.S. Treasury Bond 1-3 Year Index',
       'S&P Canada Treasury Bill Index',
       'S&P U.S. Treasury Bond Current 30-Year Index',
       'S&P U.S. Treasury Bond Curren

In [8]:
def compute_means(X, df_mean):
    # Compute means of X
    means = X.mean()
    
    # Case if ALL values of at least one feature of X are NaN, then reaplace with the whole df_mean
    if means.isna().sum() != 0:
        means.fillna(df_mean, inplace=True)
        
    return means

def split_subsample_sequence(df, length, df_mean=None):
    '''Return one single random sample (X_sample, y_sample) containing one sequence each of length `length`'''
    # Trick to save time during potential recursive calls
    if df_mean is None:
        df_mean = df.mean()

    df_subsample = subsample_sequence(df, length)
    
    y_sample = df_subsample.iloc[length-1]['volume_gross']
    # Case y_sample is NaN: redraw !
    if y_sample != y_sample: # A value is not equal to itself only for NaN
            X_sample, y_sample = split_subsample_sequence(df, length, df_mean) # Recursive call !!!
            return np.array(X_sample), np.array(y_sample)

    X_sample = df_subsample[0:length-1]
    # Case X_sample has some NaNs
    if X_sample.isna().sum().sum() !=0:
        X_sample = X_sample.fillna(compute_means(X_sample, df_mean))
        X_sample = X_sample.values
        
    return np.array(X_sample), np.array(y_sample)

In [9]:
def get_X_y(df, n_sequences, length):
    '''Return a list of samples (X, y)'''
    # $CHALLENGIFY_BEGIN
    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(df, length)
        X.append(xi)
        y.append(yi)
        
    X = np.array(X)
    y = np.array(y)
    # $CHALLENGIFY_END
    return X, y

# Modelling

In [33]:
len_ = int(0.9*orig_df.shape[0])
df_train = orig_df[:len_]
df_test = orig_df[len_:]

In [34]:
scaler = MinMaxScaler()

scaler.fit(df_train)

X_train = scaler.transform(df_train)
X_test = scaler.transform(df_test)

In [37]:
X_train, y_train = get_X_y(df_train, 900, 61)
X_test, y_test = get_X_y(df_test, 100, 61)

In [38]:
def custom_neg_mape(y_true, y_pred):
   
    return -MAPE(y_true, y_pred)

In [39]:
def init_model():
    mtr = custom_neg_mape #neg MAPE
    opt = optimizers.RMSprop(learning_rate=0.001)
    
    model = Sequential()
    
    model.add(layers.LSTM(20, return_sequences=False, activation='tanh', input_shape=(60, 58)))
    model.add(layers.Dense(32))
    
    #model.add(layers.LSTM(20, activation='tanh'))
    #model.add(Dropout(0.2))
    
    model.add(layers.Dense(1, activation='linear'))
    
    model.compile(loss='mse', 
                  optimizer=opt, 
                  metrics=[mtr])
    
    return model

In [40]:
model = init_model()
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 20)                6320      
                                                                 
 dense_4 (Dense)             (None, 32)                672       
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 7,025
Trainable params: 7,025
Non-trainable params: 0
_________________________________________________________________


In [41]:
es = EarlyStopping(monitor='val_loss', verbose=1, patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train,
            validation_split=0.2,
            epochs=100, 
            batch_size=16,
            verbose=1)

Epoch 1/100
45/45 [==============================] - 4s 45ms/step - loss: 564294346904436736.0000 - custom_neg_mape: -100.0000 - val_loss: 707043701020426240.0000 - val_custom_neg_mape: -100.0000
Epoch 2/100
45/45 [==============================] - 1s 32ms/step - loss: 564294278184960000.0000 - custom_neg_mape: -100.0000 - val_loss: 707043701020426240.0000 - val_custom_neg_mape: -100.0000
Epoch 3/100
45/45 [==============================] - 1s 31ms/step - loss: 564294449983651840.0000 - custom_neg_mape: -100.0000 - val_loss: 707043701020426240.0000 - val_custom_neg_mape: -100.0000
Epoch 4/100
45/45 [==============================] - 1s 30ms/step - loss: 564294484343390208.0000 - custom_neg_mape: -100.0000 - val_loss: 707043701020426240.0000 - val_custom_neg_mape: -100.0000
Epoch 5/100
45/45 [==============================] - 1s 30ms/step - loss: 564294381264175104.0000 - custom_neg_mape: -100.0000 - val_loss: 707043701020426240.0000 - val_custom_neg_mape: -100.0000
Epoch 6/100
45/45 [=

45/45 [==============================] - 1s 31ms/step - loss: 564293865868099584.0000 - custom_neg_mape: -99.9979 - val_loss: 707043082545135616.0000 - val_custom_neg_mape: -99.9980
Epoch 44/100
45/45 [==============================] - 1s 31ms/step - loss: 564293865868099584.0000 - custom_neg_mape: -99.9978 - val_loss: 707043082545135616.0000 - val_custom_neg_mape: -99.9979
Epoch 45/100
45/45 [==============================] - 1s 32ms/step - loss: 564293797148622848.0000 - custom_neg_mape: -99.9978 - val_loss: 707043082545135616.0000 - val_custom_neg_mape: -99.9978
Epoch 46/100
45/45 [==============================] - 1s 30ms/step - loss: 564293694069407744.0000 - custom_neg_mape: -99.9977 - val_loss: 707043082545135616.0000 - val_custom_neg_mape: -99.9977
Epoch 47/100
45/45 [==============================] - 1s 30ms/step - loss: 564293762788884480.0000 - custom_neg_mape: -99.9976 - val_loss: 707043013825658880.0000 - val_custom_neg_mape: -99.9976
Epoch 48/100
45/45 [==================

KeyboardInterrupt: 